In [6]:
import os
import numpy as np # linear algebra
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import PIL
import PIL.Image
from tensorflow import keras
import tensorflow_datasets as tfds
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [12]:
def train_test_animals():
# Creates an instance of an ImageDataGenerator called train_datagen, and a train_generator, train_datagen.flow_from_directory
  from tensorflow.keras.preprocessing.image import ImageDataGenerator
  #splits data into training and testing(validation) sets
  train_datagen =ImageDataGenerator(rescale=1./255, validation_split=0.25)
  import matplotlib.pyplot as plt

  images_per_class_train = 750
  images_per_class_test = 250

  #training data
  train_generator = train_datagen.flow_from_directory('/content/CNN_MultiClass_data/animals',
  # Source directory
    target_size=(150, 150), # Resizes images
    batch_size=images_per_class_train,
    class_mode='categorical',subset = 'training', shuffle=True)
  #Testing data
  validation_generator = train_datagen.flow_from_directory('/content/CNN_MultiClass_data/validation',
    target_size=(150, 150),
    batch_size=images_per_class_test,
    class_mode='categorical',
    subset='validation', shuffle=True) # set as validation data

  # Lấy ảnh và nhãn từ tập dữ liệu
  train_images, labels_train = next(train_generator)
  test_images, labels_test = next(validation_generator)

  return train_images, test_images, labels_train, labels_test

X_train, X_test, y_train, y_test = train_test_animals()

Found 2250 images belonging to 3 classes.
Found 750 images belonging to 3 classes.


Đưa ảnh về dạng vecto

In [13]:
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

y_train = np.argmax(y_train, axis=1)
y_test = np.argmax(y_test, axis=1)

#Logistic Regression

In [14]:
model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.516


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


#ANN
Giảm dữ liệu xuống 225 chiều

In [15]:
from sklearn.decomposition import PCA

pca = PCA(n_components=225)
pca.fit(X_train)

X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

In [16]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

model = Sequential()

model.add(Flatten(input_shape = (225,)))
model.add(Dense(128,activation = 'relu'))
model.add(Dense(64,activation = 'relu'))
model.add(Dense(32,activation = 'relu'))
model.add(Dense(10,activation = 'softmax'))

model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = 'Adam',
              metrics = ['accuracy'])

model.fit(X_train_pca,y_train,epochs= 4 , validation_split = .2)

Epoch 1/4
19/19 [==============================] - 4s 58ms/step - loss: 1.9900 - accuracy: 0.2867 - val_loss: 1.4368 - val_accuracy: 0.3800
Epoch 2/4
19/19 [==============================] - 0s 14ms/step - loss: 1.0056 - accuracy: 0.5700 - val_loss: 1.2412 - val_accuracy: 0.4200
Epoch 3/4
19/19 [==============================] - 0s 16ms/step - loss: 0.6510 - accuracy: 0.7617 - val_loss: 1.2068 - val_accuracy: 0.4600
Epoch 4/4
19/19 [==============================] - 0s 12ms/step - loss: 0.4626 - accuracy: 0.8567 - val_loss: 1.2562 - val_accuracy: 0.4533


In [17]:
y_pred = model.predict(X_test_pca)
y_pred = y_pred.argmax(axis = 1)
print(y_pred)

accuracy = accuracy_score(y_pred, y_pred)
print("Accuracy:", accuracy)

8/8 [==============================] - 0s 5ms/step
[1 0 0 1 0 0 0 1 1 1 2 1 2 2 2 0 1 2 2 0 0 0 0 0 1 2 0 0 2 1 2 0 2 1 0 0 1
 0 0 2 2 2 0 2 0 2 0 0 1 0 2 0 0 2 2 2 0 1 2 0 0 0 2 0 1 0 1 2 1 1 2 1 0 1
 2 0 1 0 0 1 0 0 2 0 0 0 2 1 0 1 2 1 2 0 1 0 1 0 0 2 2 1 0 0 0 2 0 1 0 2 1
 0 0 2 2 1 2 1 1 2 0 2 1 2 0 0 0 2 0 2 2 2 0 0 2 0 1 0 1 1 1 2 0 2 2 2 0 1
 0 2 0 1 1 2 0 0 1 0 0 0 0 1 0 2 0 0 0 1 0 0 0 0 0 1 1 1 0 2 0 1 2 1 1 2 2
 1 0 2 1 0 0 1 0 0 1 2 1 2 1 2 1 2 1 1 0 0 1 0 2 2 1 0 0 2 1 1 0 2 0 1 2 1
 0 0 0 2 1 2 1 1 0 1 1 2 0 2 1 1 0 2 0 1 1 1 0 0 2 2 2 1]
Accuracy: 1.0


#CNN

In [20]:
def train_test_animals():
# Creates an instance of an ImageDataGenerator called train_datagen, and a train_generator, train_datagen.flow_from_directory
  from tensorflow.keras.preprocessing.image import ImageDataGenerator
  #splits data into training and testing(validation) sets
  train_datagen =ImageDataGenerator(rescale=1./255, validation_split=0.25)
  import matplotlib.pyplot as plt

  images_per_class_train = 750
  images_per_class_test = 250

  #training data
  train_generator = train_datagen.flow_from_directory('/content/CNN_MultiClass_data/animals',
  # Source directory
    target_size=(150, 150), # Resizes images
    batch_size=images_per_class_train,
    class_mode='categorical',subset = 'training', shuffle=True)
  #Testing data
  validation_generator = train_datagen.flow_from_directory('/content/CNN_MultiClass_data/validation',
    target_size=(150, 150),
    batch_size=images_per_class_test,
    class_mode='categorical',
    subset='validation', shuffle=True) # set as validation data

  # Lấy ảnh và nhãn từ tập dữ liệu
  train_images, labels_train = next(train_generator)
  test_images, labels_test = next(validation_generator)

  return train_images, test_images, labels_train, labels_test

X_train, X_test, y_train, y_test = train_test_animals()

Found 2250 images belonging to 3 classes.
Found 750 images belonging to 3 classes.


Chuẩn bị dữ liệu train, test để phân loại 2 lớp chó, mèo

In [21]:
def get_data(images, labels, class_indices, num_images):
  filtered_images = []
  filtered_labels = []
  for i, label in enumerate(labels):
    if label[class_indices[0]] == 1 or label[class_indices[1]] == 1:
      filtered_images.append(images[i])
      if label[class_indices[0]] == 1:
        filtered_labels.append(0)  # Nhãn cho lớp 0
      else:
        filtered_labels.append(1)  # Nhãn cho lớp 1
      num_images -= 1
      if num_images == 0:
        break
  return np.array(filtered_images), np.array(filtered_labels)

class_indices = [0, 1]

X_train, y_train = get_data(X_train, y_train, class_indices, 150)

X_test, y_test = get_data(X_test, y_test, class_indices, 100)

In [22]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Dropout(0.3),  # Tầng Dropout với tỷ lệ dropout là 0.3
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.4),  # Tầng Dropout với tỷ lệ dropout là 0.4
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.5),  # Tầng Dropout với tỷ lệ dropout là 0.5
    Flatten(),
    Dense(64, activation='relu'),
    Dense(2, activation='softmax')  # 2 lớp đầu ra với softmax activation
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [23]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))
predictions = model.predict(X_test)

Epoch 1/10
5/5 [==============================] - 10s 2s/step - loss: 1.9196 - accuracy: 0.4733 - val_loss: 0.6939 - val_accuracy: 0.4200
Epoch 2/10
5/5 [==============================] - 9s 2s/step - loss: 0.6944 - accuracy: 0.5000 - val_loss: 0.6934 - val_accuracy: 0.4900
Epoch 3/10
5/5 [==============================] - 7s 1s/step - loss: 0.6915 - accuracy: 0.5400 - val_loss: 0.6933 - val_accuracy: 0.4900
Epoch 4/10
5/5 [==============================] - 9s 2s/step - loss: 0.6907 - accuracy: 0.5400 - val_loss: 0.6934 - val_accuracy: 0.4900
Epoch 5/10
5/5 [==============================] - 7s 2s/step - loss: 0.6901 - accuracy: 0.5400 - val_loss: 0.6935 - val_accuracy: 0.4900
Epoch 6/10
5/5 [==============================] - 8s 2s/step - loss: 0.6922 - accuracy: 0.5400 - val_loss: 0.6941 - val_accuracy: 0.4900
Epoch 7/10
5/5 [==============================] - 9s 2s/step - loss: 0.6909 - accuracy: 0.5400 - val_loss: 0.6943 - val_accuracy: 0.4900
Epoch 8/10
5/5 [========================

In [24]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(2, activation='softmax')  # 2 lớp đầu ra với softmax activation
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [25]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

predictions = model.predict(X_test)

Epoch 1/10
5/5 [==============================] - 9s 2s/step - loss: 0.8904 - accuracy: 0.4667 - val_loss: 0.6933 - val_accuracy: 0.5100
Epoch 2/10
5/5 [==============================] - 8s 2s/step - loss: 0.6976 - accuracy: 0.4667 - val_loss: 0.6924 - val_accuracy: 0.5300
Epoch 3/10
5/5 [==============================] - 7s 1s/step - loss: 0.7006 - accuracy: 0.5133 - val_loss: 0.6928 - val_accuracy: 0.5300
Epoch 4/10
5/5 [==============================] - 8s 2s/step - loss: 0.6897 - accuracy: 0.6467 - val_loss: 0.6929 - val_accuracy: 0.4900
Epoch 5/10
5/5 [==============================] - 7s 1s/step - loss: 0.6854 - accuracy: 0.6333 - val_loss: 0.6915 - val_accuracy: 0.5200
Epoch 6/10
5/5 [==============================] - 8s 2s/step - loss: 0.6682 - accuracy: 0.6133 - val_loss: 0.6817 - val_accuracy: 0.5800
Epoch 7/10
5/5 [==============================] - 7s 1s/step - loss: 0.6319 - accuracy: 0.7067 - val_loss: 0.7164 - val_accuracy: 0.5000
Epoch 8/10
5/5 [=========================